# PyTorch Used Cars Dataset Example


## 1. Introduction

This tutorial focuses on how to train a federated regression model on Non-IID dataset using PyTorch framework. We will be working on the Used Cars dataset to perform federated learning. The sections of this tutorial are presented as follows;

- Dataset Preparation;
- Node Configurations;
- Create an Experiment to Train a Model;
- Testing Federated Model.

## 2. Dataset Preparation

In this tutorial, we will be using the Used Cars dataset. The goal of the model will be to predict the price of the car based on given features. 

You can download the dataset from [here](https://www.kaggle.com/adityadesai13/used-car-dataset-ford-and-mercedes). To be able to download this dataset you need to have a Kaggle account. After downloading, you can create folders for the dataset in the Fed-BioMed `Researcher` `fbm-researcher/notebooks/data` directory.

```shell
cd /path/to/fedbiomed/directory
fedbiomed component create -c researcher
mkdir fbm-researcher/notebooks/data/UsedCars && mkdir fbm-researcher/notebooks/data/UsedCars/raw 
```

You can extract CSV files in the zip file into `fbm-researcher/notebooks/data/UsedCars/raw`. Your file tree should be like the tree below;

```shell
├── data
│   └── UsedCars
│       └── raw
│           ├── audi.csv
│           ├── bmw.csv
│           ├── cclass.csv
│           ├── focus.csv
│           ├── ford.csv
│           ├── hyundi.csv
│           ├── merc.csv
│           ├── skoda.csv
│           ├── toyota.csv
│           ├── unclean cclass.csv
│           ├── unclean focus.csv
│           ├── vauxhall.csv
│           └── vw.csv

```

### 2.1 Selecting CSV Dataset for Each Node

Each CSV dataset contains features for different car brands. It is a good example for applying federated learning through each dataset if we assume that these datasets will be stored in different locations. We will be working on 3 datasets that are `audi.csv`, `bmw.csv`, and `ford.csv`. We will deploy `audi.csv` and `bmw.csv` on different nodes and use `ford.csv` for final testing at the central researcher using the model trained on two nodes.   

### 2.2 Preprocessing 

Before deploying datasets we need to apply some preprocessing to make them ready for the federated training. Since car `model` and `fuelType` features are not consistent across the dataset, we can drop them. We also need to apply label encoding for the `transmission` feature.

**Note:** Dropping and encoding columns can be also done in the `training_data` method of `TrainingPlan` but it is always better to deploy clean/prepared datasets in the nodes. 
 

Let's starting with loading CSV datasets using `pd.read_csv` API. Please make sure that you have launched your Jupyter notebook using the command `fedbiomed researcher start` so you can follow code examples without changing file paths.  


In [1]:
import os
import pandas as pd
from fedbiomed.researcher.config import config
# Optional: to be able use different researcher configuration.
# config.load(root=<different-component-root>) 


root_dir = config.root

audi = pd.read_csv(os.path.join(root_dir,'notebooks','data','UsedCars', 'raw', 'audi.csv'))
bmw = pd.read_csv(os.path.join(root_dir,'notebooks', 'data', 'UsedCars', 'raw', 'bmw.csv'))
ford = pd.read_csv(os.path.join(root_dir,'notebooks', 'data', 'UsedCars', 'raw', "ford.csv"))

2025-10-15 14:20:09,153 fedbiomed WARNING - Found version 3, expected version 3.1.0 -> See https://fedbiomed.org/latest/user-guide/deployment/versions for more information

2025-10-15 14:20:09,153 fedbiomed WARNING - DEPRECATION: You are using an old configuration file for researcher. Please add 'node_disconnection_timeout' value in `server` section of the researcher configuration.

2025-10-15 14:20:09,163 fedbiomed WARNING - Found version 3, expected version 3.1.0 -> See https://fedbiomed.org/latest/user-guide/deployment/versions for more information

Drop columns for car `model` & `fuelType` as labels are not consistent across files. 

In [2]:
audi.drop(columns = ['model','fuelType'], inplace = True)
bmw.drop(columns = ['model','fuelType'], inplace = True)
ford.drop(columns = ['model','fuelType'], inplace = True)

Label encoding for `transmission` column

In [3]:
audi['transmission'] = audi['transmission'].map({'Automatic':0,'Manual':1,'Semi-Auto':2,'Other':3})
bmw['transmission'] = bmw['transmission'].map({'Automatic':0,'Manual':1,'Semi-Auto':2,'Other':3})
ford['transmission'] = ford['transmission'].map({'Automatic':0,'Manual':1,'Semi-Auto':2,'Other':3})

Now, we can save our new CSV files into `data/UsedCars` directory

In [4]:
audi.to_csv(os.path.join(root_dir, 'notebooks', 'data', 'UsedCars', 'audi_transformed.csv'),header = True,index= False)
bmw.to_csv(os.path.join(root_dir, 'notebooks', 'data', 'UsedCars', 'bmw_transformed.csv'),header = True,index= False)
ford.to_csv(os.path.join(root_dir, 'notebooks', 'data', 'UsedCars', 'ford_transformed.csv'),header = True,index= False)

## 3. Node Configurations

We will deploy the `audi_transformed.csv` and `bmw_transformed.csv` datasets on different nodes.  

1. **Configuring First Node**
    * Run `fedbiomed component create -c node --path ./node-audi`
    * Move `audi_transformed.csv` from `fbm-researcher/notebooks/data/UsedCars` to the `Node` `node-audi/data` with the following command: 
        - `mv fbm-researcher/notebooks/data/UsedCars/audi_transformed.csv node-audi/data/`
    * Run `fedbiomed node --path ./node-audi dataset add`
    * Select option 1 to add a csv file (audi_transformed.csv)
    * Choose a name for dataset, For example `Used-Cars-Audi`
    * Choose tag for the dataset. This part is important because we will be sending search request to nodes with this 
    specified tag. Therefore please type `#UsedCars` and enter.
    * Enter a description for the dataset 
    * Select the audi_transformed.csv file in the file selection window
  
   
2. **Configuring Second Node**
    * Run `fedbiomed component create -c node --path ./node-bmw`
    * Move `bmw_transformed.csv` from `fbm-researcher/notebooks/data/UsedCars` to the `Node` `node-bmw/data` with the following command: 
        - `mv fbm-researcher/notebooks/data/UsedCars/bmw_transformed.csv node-bmw/data/`
    * Run `fedbiomed node --path ./node-bmw dataset add`
    * Select option 1 to add a csv file (bmw_transformed.csv)
    * Choose a name for dataset, For example `Used-Cars-BMW`
    * Since we entered the tag as `#UsedCars`, we need to use the same tag for this one too. 
    * Enter a description for the dataset
    * Select the bmw_trasnformed.csv file in the file selection window
    
3. **Starting Nodes**
    Please run the following command to start the node that has the `audi` dataset.
    
    ```shell
    fedbiomed node --path ./node-audi start
    ```
    Please open a new terminal window to start the node that has the `bmw` dataset 
    
    ```shell
    fedbiomed node --path ./node-bmw start
    ```

## 4. Create an Experiment to Train a Model 

Before creating an experiment, we need to define training plan class.

In [51]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F

from fedbiomed.common.dataset import TabularDataset 
from fedbiomed.common.training_plans import TorchTrainingPlan
from fedbiomed.common.datamanager import DataManager


# Here we define the training plan to be used for the experiment.
class MyTrainingPlan(TorchTrainingPlan):

    # Model
    def init_model(self):
        model_args = self.model_args()
        model = self.Net(model_args)
        return model

    # Dependencies
    def init_dependencies(self):
        deps = ["from fedbiomed.common.dataset import TabularDataset",
                "import pandas as pd"]
        return deps

    # network
    class Net(nn.Module):
        def __init__(self, model_args):
            super().__init__()
            self.in_features = model_args['in_features']
            self.out_features = model_args['out_features']
            self.fc1 = nn.Linear(self.in_features, 5)
            self.fc2 = nn.Linear(5, self.out_features)

        def forward(self, x):
            x = self.fc1(x)
            x = F.relu(x)
            x = self.fc2(x)
            return x

    def training_step(self, data, target):
        output = self.model().forward(data).float()
        criterion = torch.nn.MSELoss()
        loss   = torch.sqrt(criterion(output, target.unsqueeze(1)))
        return loss

    # class csv_Dataset(Dataset):
    # # Here we define a custom Dataset class inherited from the general torch Dataset class
    # # This class takes as argument a .csv file path and creates a torch Dataset
    #     def __init__(self, dataset_path, x_dim):
    #         self.input_file = pd.read_csv(dataset_path,sep=',',index_col=False)
    #         x_train = self.input_file.loc[:,('year','transmission','mileage','tax','mpg','engineSize')].values
    #         y_train = self.input_file.loc[:,'price'].values
    #         self.X_train = torch.from_numpy(x_train).float()
    #         self.Y_train = torch.from_numpy(y_train).float()

    #     def __len__(self):
    #         return len(self.Y_train)

    #     def __getitem__(self, idx):
    #         return (self.X_train[idx], self.Y_train[idx])

    def training_data(self):
    # The training_data creates the Dataloader to be used for training in the general class TorchTrainingPlan of fedbiomed
        dataset = TabularDataset(
            input_columns=['year','transmission','mileage','tax','mpg','engineSize'], 
            target_columns=['price'],
            transform=lambda xs: torch.as_tensor(xs, dtype=torch.float32),
            target_transform=lambda xs: torch.as_tensor(xs, dtype=torch.float32)
        )
        return DataManager(dataset=dataset)


### 4.1 Defining Arguments for The Experiment
An experiment is a class that orchestrates the training processes that run on different nodes. The experiment has to be initialized with necessary arguments to inform nodes about the training plan. In this case, first, you need to define `model_arg`, `training_args`, `tags`, and `round`.

Please visit [experiment documentation](../../../user-guide/researcher/experiment/) to get detailed information about the experiment class.

In [52]:
# model parameters
model_args = {
    'in_features': 6,
    'out_features': 1
}

# training parameters
training_args = {
    'loader_args': { 'batch_size': 40, 'shuffle': True },
    'optimizer_args': {
          'lr': 1e-3
    },
    'epochs': 2,
}

The other arguments that should be passed to the experiment is the training plan class which is `MyTrainingPlan`. 

You should also indicate which method should be chosen to aggregate model parameters after every round. The basic federation scheme is federated averaging, implemented in Fed-BioMed in the class `FedAverage`. You can also visit [aggregation documentation](../../../user-guide/researcher/aggregation/) to have more information about aggregation process.

Since we are going to use every `Node` that has `UsedCars` datasets, the `node_selection_strategy` should be `None` which means that every `Node` will be part of the federated training. 

### 4.2 What happens during the initialization of an experiment?

1. The experiment searches for nodes that have datasets that have been saved with the `#UsedCars` tag.
2. The experiment is set up to manage the training process across the nodes with the given arguments

**Note**: It is possible to send search requests to only specified nodes with the `Nodes` argument of the experiment. Please visit [listing datasets and selecting nodes](../../../user-guide/researcher/listing-datasets-and-selecting-nodes) documentation for more information.

In [53]:
from fedbiomed.researcher.federated_workflows import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

tags =  ['#UsedCars']
rounds = 5

exp = Experiment(tags=tags,
                 training_plan_class=MyTrainingPlan,
                 model_args=model_args,
                 training_args=training_args,
                 round_limit=rounds,
                 aggregator=FedAverage(),
                 node_selection_strategy=None)

2025-10-15 14:40:03,032 fedbiomed INFO - Updating training data. This action will update FederatedDataset, and the nodes that will participate to the experiment.

2025-10-15 14:40:03,042 fedbiomed DEBUG - Node: NODE_c6b00f31-ab90-4137-90b1-e84da620ae60 polling for the tasks

2025-10-15 14:40:03,045 fedbiomed DEBUG - Node: NODE_84388ee9-396b-42e9-ba5f-e72a4af75fb9 polling for the tasks

2025-10-15 14:40:03,048 fedbiomed INFO - Node selected for training -> Default Node Name
Node ID is -> NODE_c6b00f31-ab90-4137-90b1-e84da620ae60

2025-10-15 14:40:03,050 fedbiomed INFO - Node selected for training -> Default Node Name
Node ID is -> NODE_84388ee9-396b-42e9-ba5f-e72a4af75fb9

<function extract_symbols at 0x71388a9fda20>


2025-10-15 14:40:03,066 fedbiomed DEBUG - Model file has been saved: /mnt/c/Users/adincer/Desktop/fedbiomed/fbm-researcher/var/experiments/Experiment_0133/model_e573251d-f061-4620-80e3-512e4e34b97d.py

The experiment also receives loss values during training on each node. In Fed-BioMed, it is possible to use a tensorboard to display loss values during training. Please refer to Fed-BioMed's [tensorboard documentation](../../../user-guide/researcher/tensorboard) to enable tensorboard.

Let's start the experiment. By default, this function doesn't stop until all the `round_limit` rounds are done for all the nodes.

In [54]:
exp.run()

2025-10-15 14:40:04,012 fedbiomed INFO - Sampled nodes in round 0 ['NODE_c6b00f31-ab90-4137-90b1-e84da620ae60', 'NODE_84388ee9-396b-42e9-ba5f-e72a4af75fb9']

<function extract_symbols at 0x71388a9fda20>


2025-10-15 14:40:04,022 fedbiomed INFO - Sending request 
					 To: NODE_c6b00f31-ab90-4137-90b1-e84da620ae60 
					 Request: : TRAIN
 -----------------------------------------------------------------

2025-10-15 14:40:04,023 fedbiomed INFO - Sending request 
					 To: NODE_84388ee9-396b-42e9-ba5f-e72a4af75fb9 
					 Request: : TRAIN
 -----------------------------------------------------------------

2025-10-15 14:40:04,041 fedbiomed DEBUG - Node: NODE_c6b00f31-ab90-4137-90b1-e84da620ae60 polling for the tasks

...

2025-10-15 14:43:35,153 fedbiomed INFO - TRAINING 
					 NODE_ID: NODE_84388ee9-396b-42e9-ba5f-e72a4af75fb9 
					 Node Name: Default Node Name 
					 Round 5 Epoch: 2 | Iteration: 260/270 (96%) | Samples: 10400/10800
 					 Loss: 10128.603516 
					 ---------

2025-10-15 14:43:35,194 fedbiomed INFO - TRAINING 
					 NODE_ID: NODE_c6b00f31-ab90-4137-90b1-e84da620ae60 
					 Node Name: Default Node Name 
					 Round 5 Epoch: 2 | Iteration: 267/267 (100%) | Samples: 10668/10668
 					 Loss: 12757.570312 
					 ---------

2025-10-15 14:43:35,774 fedbiomed INFO - TRAINING 
					 NODE_ID: NODE_84388ee9-396b-42e9-ba5f-e72a4af75fb9 
					 Node Name: Default Node Name 
					 Round 5 Epoch: 2 | Iteration: 270/270 (100%) | Samples: 10781/10781
 					 Loss: 15258.650391 
					 ---------

2025-10-15 14:43:35,797 fedbiomed INFO - Nodes that successfully reply in round 4 ['NODE_c6b00f31-ab90-4137-90b1-e84da620ae60', 'NODE_84388ee9-396b-42e9-ba5f-e72a4af75fb9']

5

Save trained model to file

In [55]:
exp.training_plan().export_model('./trained_model')

### 4.3 Extracting Training Results

#### Timing 

Training replies for each round are available in `exp.training_replies()` (index 0 to (`rounds` - 1) ). You can display the keys of each round by running the following script. 

In [56]:
print("\nList the training rounds : ", exp.training_replies().keys())


List the training rounds :  dict_keys([0, 1, 2, 3, 4])


Now, let's see how training details can be accessed via `training_replies()`. The following parameters will be inspected;

- `rtime_training` : Real-time (clock time) spent in the training function on the node 
- `ptime_training`: Process time (user and system CPU) spent in the training function on the node
- `rtime_total`   : Real-time (clock time) spent in the researcher between sending training requests and handling the responses

_Note: The following code accesses the training replies of the last round of the experiment._

In [57]:
print("\nList the training rounds : ", exp.training_replies().keys())

print("\nList the nodes for the last training round and their timings : ")
round_data = exp.training_replies()[rounds - 1]
for r in round_data.values():
    print("\t- {id} :\
    \n\t\trtime_training={rtraining:.2f} seconds\
    \n\t\tptime_training={ptraining:.2f} seconds\
    \n\t\trtime_total={rtotal:.2f} seconds".format(id = r['node_id'],
        rtraining = r['timing']['rtime_training'],
        ptraining = r['timing']['ptime_training'],
        rtotal = r['timing']['rtime_total']))
print('\n')


List the training rounds :  dict_keys([0, 1, 2, 3, 4])

List the nodes for the last training round and their timings : 
	- NODE_c6b00f31-ab90-4137-90b1-e84da620ae60 :    
		rtime_training=41.26 seconds    
		ptime_training=94.20 seconds    
		rtime_total=41.92 seconds
	- NODE_84388ee9-396b-42e9-ba5f-e72a4af75fb9 :    
		rtime_training=41.84 seconds    
		ptime_training=95.81 seconds    
		rtime_total=41.92 seconds




#### Federated Parameters
Federated model parameters for each round are available in `exp.aggregated_params()` (index 0 to (`rounds` - 1) ).
For example, you can easily view the federated parameters for the last round of the experiment:

In [58]:
print("\nList the training rounds : ", exp.aggregated_params().keys())
print("\nAccess the federated params for the last training round :")
print("\t- parameter data: ", exp.aggregated_params()[rounds - 1]['params'].keys())



List the training rounds :  dict_keys([0, 1, 2, 3, 4])

Access the federated params for the last training round :
	- parameter data:  dict_keys(['fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias'])


In [59]:
exp.training_plan().model()

Net(
  (fc1): Linear(in_features=6, out_features=5, bias=True)
  (fc2): Linear(in_features=5, out_features=1, bias=True)
)

## 5. Testing Federated Model

In this section, we will create a test function to obtain RMSE on `ford_transformed.csv` dataset by using federated model. 

### 5.1 Aggregated Parameters 

`model_instance` returns the model that we have created in the previous section. You can load specific aggregated parameters which are obtained in the round. Thereafter, it will make the predictions using those parameters. The last round gives the last aggregated model parameters which represents the final model.   

In [60]:
fed_model = exp.training_plan().model()
fed_model.load_state_dict(exp.aggregated_params()[rounds - 1]['params'])

<All keys matched successfully>

In [61]:
 fed_model

Net(
  (fc1): Linear(in_features=6, out_features=5, bias=True)
  (fc2): Linear(in_features=5, out_features=1, bias=True)
)

In [62]:
test_dataset_path = os.path.join(root_dir, 'notebooks', 'data', 'UsedCars', 'ford_transformed.csv')

### 5.2 Creating A Test Function

Let's create a test function that returns `rmse`. 

In [72]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import pandas as pd

def cal_rmse(actual, prediction):
    return ((actual- prediction)**2).mean()**0.5

def testing_rmse(model, data_loader):
    model.eval()
    test_loss = 0
    correct = 0
    device = 'cpu'
    preds = []
    targets = []
    with torch.no_grad():
        for data, target in data_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            preds.append(output.numpy().flatten())
            targets.append(target.numpy().flatten())
    # rmse = cal_rmse(data_loader.dataset.Y_train.numpy(),np.hstack(preds))
    rmse = cal_rmse(targets,np.hstack(preds))
    return rmse

We also need to create a Dataset class for PyTorch data loader. 

In [73]:
# class csv_Dataset(Dataset):
#         def __init__(self, dataset_path):
#             self.input_file = pd.read_csv(dataset_path,sep=',',index_col=False)
#             x_train = self.input_file.loc[:,('year','transmission','mileage','tax','mpg','engineSize')].values
#             y_train = self.input_file.loc[:,'price'].values
#             self.X_train = torch.from_numpy(x_train).float()
#             self.Y_train = torch.from_numpy(y_train).float()

#         def __len__(self):            
#             return len(self.Y_train)

#         def __getitem__(self, idx):

#             return (self.X_train[idx], self.Y_train[idx])

In [74]:
from fedbiomed.common.dataset_types import DataReturnFormat

# dataset = csv_Dataset(test_dataset_path)
dataset = TabularDataset(
            input_columns=['year','transmission','mileage','tax','mpg','engineSize'], 
            target_columns=['price'],
            transform=lambda xs: torch.as_tensor(xs, dtype=torch.float32),
            target_transform=lambda xs: torch.as_tensor(xs, dtype=torch.float32)
        )
dataset.complete_initialization(controller_kwargs={'root':test_dataset_path}, to_format=DataReturnFormat.TORCH)
train_kwargs = { 'shuffle': True}
data_loader = DataLoader(dataset, **train_kwargs)

In [75]:
rmse = testing_rmse(fed_model, data_loader)
print(rmse)

11358.927414153151


### 5.3 Plotting RMSE Values of Each Round

In [76]:
!pip install matplotlib

I0000 00:00:1760533386.587160   36789 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 31.0 MB/s eta 0:00:0000:0100:01
  Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 58.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.9/113.9 kB 30.3 MB/s eta 0:00:00
  Using cached contourpy-1.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (325 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 60.6 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [ ]:
errors = []

for i in range(rounds):
    fed_model = exp.training_plan().model()
    fed_model.load_state_dict(exp.aggregated_params()[i]['params'])
    loss = testing_rmse(fed_model, data_loader)
    errors.append(loss)

In [ ]:
### Plotting 
import matplotlib.pyplot as plt
plt.plot(errors, label = 'Federated Test Loss')
plt.xlabel('Round')
plt.ylabel('Loss')
plt.legend()